# FHN equations

$$
\begin{align}
\partial_t u &= D_u \Delta u + [u(1-u)(u-a) -v  ]/\epsilon\\
\partial_t v &= D_v \Delta v + \epsilon (u - \gamma v)
\end{align}
$$

$$
(a, \epsilon, \gamma, D_u, D_v) 
= (0.25, 0.05, 3.0, 4.0 \times 10^{-5}, 1.0\times 10^{-7})
$$

## Importing modules 

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import equations as eq

import h5py
import time

import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Setting Parameters 

In [3]:
NW = 100
fhn01 = eq.FHN(NW)
args = fhn01.getParamDefault()

J = fhn01.J
x = fhn01.sc.get_x()

trange = np.linspace(0.0, 50.0, 2501)

## Setting initial data 

In [4]:
uv0p = fhn01.sc.get_zero_pwave((2,))
uv0p[0] = np.zeros(J) + 0.4*np.exp(-800.0*(x-0.4)**2)
uv0p[1] = np.zeros(J) + 5.0e-2*np.exp(-800.0*(x-0.35)**2)
uv0c = fhn01.sc.transform_wp2wc(uv0p)

## Computation 

In [5]:
max_step = 1.0e-2
with h5py.File('fhn.hdf5', 'w') as fh:
    fhn01.mkInitDataSet(uv0c, fh)    
    fhn01.evolve(fh,trange,args, max_step=max_step, method='BDF', prog_bar=True)
    fhn01.evolve(fh,trange,args, max_step=max_step, method='BDF', prog_bar=False)

Computing ...: 100%|██████████| 25/25 [00:31<00:00,  1.24s/it]


## Visualization 

In [6]:
with h5py.File('fhn.hdf5', 'r') as fh:
    wp = fh['wp'][()]

In [7]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

ax.set_ylim((-0.5, 1.5))
ax.set_xlim((-0.1, 1.1))
ax.grid(True)
lines = plt.plot(x, [np.nan]*len(x), x, [np.nan]*len(x));

def init():
    for l in lines:
        l.set_ydata([np.nan] * len(x))
    return lines

def animate(i):
    for j, l in enumerate(lines):
        l.set_ydata(wp[i,j,:])
    return lines

ani = animation.FuncAnimation(
    fig, animate, frames=len(trange),
    init_func=init, interval=1, blit=True,
    repeat=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …